<a href="https://colab.research.google.com/github/viktoruebelhart/Keras_NPL_News/blob/main/word_completer_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We built a Deep Learning model to classify articles into different categories. Now, a new need has emerged: our content portal wants editors and journalists to follow a consistent style when writing texts. To do this, we will create a word completer that follows a specific style, similar to the feature found in text editors.

In [1]:
import pandas as pd
import tensorflow as tf

url = 'https://github.com/allanspadini/curso-tensorflow-proxima-palavra/raw/main/dados/train.zip'

df = pd.read_csv(url, header=None, names=['ClassIndex', 'Título', 'Descrição'])

df['Texto'] = df['Título'] + '' + df['Descrição']

In [3]:
df['Texto']

,Texto
0,Wall St. Bears Claw Back Into the Black (Reute...
1,Carlyle Looks Toward Commercial Aerospace (Reu...
2,Oil and Economy Cloud Stocks' Outlook (Reuters...
3,Iraq Halts Oil Exports from Main Southern Pipe...
4,"Oil prices soar to all-time record, posing new..."
...,...
119995,Pakistan's Musharraf Says Won't Quit as Army C...
119996,Renteria signing a top-shelf dealRed Sox gener...
119997,Saban not going to Dolphins yetThe Miami Dolph...
119998,Today's NFL gamesPITTSBURGH at NY GIANTS Time:...


In [4]:
import random

random.seed(42)
df_sample = df.sample(n=1000)

We will create a corpus, which is our complete set of texts, by converting the contents of the text column to a list.

In [5]:
corpus = df_sample['Texto'].tolist()

In [14]:
from tensorflow.keras.layers import TextVectorization

Maximum vocabulary size as 20,000 words and maximum sequence size as 50 words.
These variables will be max_vocab_size and max_sequence_len, respectively.

In [20]:
max_vocab_size = 20000
max_sequence_len = 50

In [16]:
vectorizer = TextVectorization(max_tokens=max_vocab_size,
                             output_mode='int',
                             output_sequence_length=max_sequence_len)

In [17]:
vectorizer.adapt(corpus)

In [18]:
token_corpus = vectorizer(corpus)

In [19]:
import pickle

with open('vectorizer.pkl', 'wb') as file:
  pickle.dump(vectorizer, file)

With the vectorizer saved and the text tokenized, we need to organize the data into an ideal format to train the neural network, which will learn to suggest the next words. We will do this by passing different strings of text.

Let's create an empty list called input_sequences to store the text sequences. We will then populate this list based on our tokenized data. We will loop through each list of tokens in the tokenized_corpus (converted to a numeric format) and add these sequences to the input_sequences list.

In [21]:
input_sequences = []

for token_list in token_corpus.numpy():
  for i in range(1, len(token_list)):
    n_gram_sequence = token_list[:i+1]
    input_sequences.append(n_gram_sequence)

In [25]:
input_sequences

[array([536, 143]),
 array([536, 143, 151]),
 array([536, 143, 151,   8]),
 array([536, 143, 151,   8, 375]),
 array([ 536,  143,  151,    8,  375, 4691]),
 array([ 536,  143,  151,    8,  375, 4691, 3538]),
 array([ 536,  143,  151,    8,  375, 4691, 3538,    3]),
 array([ 536,  143,  151,    8,  375, 4691, 3538,    3, 8733]),
 array([ 536,  143,  151,    8,  375, 4691, 3538,    3, 8733, 4035]),
 array([ 536,  143,  151,    8,  375, 4691, 3538,    3, 8733, 4035, 5759]),
 array([ 536,  143,  151,    8,  375, 4691, 3538,    3, 8733, 4035, 5759,
        1555]),
 array([ 536,  143,  151,    8,  375, 4691, 3538,    3, 8733, 4035, 5759,
        1555,    7]),
 array([ 536,  143,  151,    8,  375, 4691, 3538,    3, 8733, 4035, 5759,
        1555,    7,  860]),
 array([ 536,  143,  151,    8,  375, 4691, 3538,    3, 8733, 4035, 5759,
        1555,    7,  860,  286]),
 array([ 536,  143,  151,    8,  375, 4691, 3538,    3, 8733, 4035, 5759,
        1555,    7,  860,  286,   37]),
 array([ 536, 

In [27]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [28]:
def prepare_sequences(sequences):
    """
    Prepares the sequences for the model by removing trailing zeros, adding left padding, truncating long sequences, and removing duplicate sequences.

    Args:
        sequences: An array of sequences (lists or NumPy arrays).

    Returns:
        A 2D NumPy array with the prepared sequences.
    """

    # Remove trailing zeros from each sequence
    sequences_without_trailing_zeros = []
    for seq in sequences:
        last_nonzero_index = np.argmax(seq[::-1] != 0)
        if last_nonzero_index == 0 and seq[-1] == 0:
            sequences_without_trailing_zeros.append(np.array([0]))
        else:
            sequences_without_trailing_zeros.append(seq[:-last_nonzero_index or None])

    # Remove duplicate sequences
    unique_sequences = []
    for seq in sequences_without_trailing_zeros:
        if seq.tolist() not in unique_sequences:  # Check if the sequence is already in the list
            unique_sequences.append(seq.tolist())  # Add to the list if it is unique

    # Find the maximum length of the sequences without trailing zeros
    max_sequence_len = max(len(seq) for seq in unique_sequences)

    # Add left padding to ensure the same length
    padded_sequences = pad_sequences(unique_sequences, maxlen=max_sequence_len, padding='pre', truncating='post')

    return padded_sequences


In [29]:
input_sequences = prepare_sequences(input_sequences)
print(input_sequences)

[[  0   0   0 ...   0 536 143]
 [  0   0   0 ... 536 143 151]
 [  0   0   0 ... 143 151   8]
 ...
 [  0   0   0 ... 252 537 306]
 [  0   0   0 ... 252 537 306]
 [  0   0   0 ... 252 537 306]]
